## Testing with the OpenAI API

In [1]:
from openai import OpenAI
from config import API_KEY, ASSISTANT_ID
client = OpenAI(api_key=API_KEY)

### Simple Usage

In [ ]:
from openai import OpenAI
from config import API_KEY
client = OpenAI(api_key=API_KEY)

completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "You are a poetic assistant, skilled in explaining complex programming concepts with creative flair."},
    {"role": "user", "content": "Compose a poem that explains the concept of recursion in programming."}
  ]
)

print(completion.choices)

[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="In the realm of code where patterns entwine,\nThere lies a concept both elegant and fine.\nRecursion, a dance of functions divine,\nA mystical loop that intertwines.\n\nA function that calls itself, oh so clever,\nAn endless cycle that seems to go on forever.\nWith each iteration, it solves a piece,\nUnraveling problems, granting release.\n\nLike a fractal, repeating patterns unfold,\nA beauty in simplicity, a story told.\nDivide and conquer, the recursive quest,\nBreaking problems down into the smallest jest.\n\nBase cases as anchors, to stop the recursion flow,\nWithout them, chaos would surely grow.\nSo mind the stack, the memory's pace,\nA balance to keep in this recursive race.\n\nSo behold the magic of recursion's embrace,\nA concept in coding, full of grace.\nIt bends the rules, defies convention,\nA loop like no other, a coder's invention.", role='assistant', function_call=None, tool_ca

In [ ]:
def get_user_prompt(role = 'user', user_prompt = ""):
  """Gets user prompt and appends location request."""
  # Append "including coordinates" to ensure location retrieval
  user_prompt += "include a dictionary at the end of your response with the following format: {place: coordinates}. Everytime you mention a place in your response, add it to the dictionary"
  message_dict = {"role": role, "content": user_prompt}
  return message_dict
    
# completion = client.chat.completions.create(model = "gpt-3.5-turbo", messages = [get_user_prompt(user_prompt="tell me about the eiffel tower")])

# print(completion.choices[0].message.content)

In [ ]:
import re

# Function to extract coordinates from text using regular expressions
# Function to extract coordinates from text using regular expressions
def extract_coordinates(text):
    # Regular expression pattern to match coordinates in the format "48.8584° N, 2.2945° E"
    pattern = r"(-?\d+\.\d+)° (N|S), (-?\d+\.\d+)° (E|W)"
    match = re.search(pattern, text)
    if match:
        latitude = float(match.group(1)) * (-1 if match.group(2) == 'S' else 1)
        longitude = float(match.group(3)) * (-1 if match.group(4) == 'W' else 1)
        return latitude, longitude
    else:
        return (-1, -1)

In [ ]:
completion = client.chat.completions.create(model = "gpt-3.5-turbo", messages = [get_user_prompt(user_prompt="tell me about the eiffel tower")])
print(f"Latitude: {extract_coordinates(completion.choices[0].message.content)[0]}\nLongitude: {extract_coordinates(completion.choices[0].message.content)[1]}")

Latitude: 48.8584
Longitude: 2.2945


![Eiffel Tower Maps](eiffeltowermaps.png)

### Multiple Locations

In [ ]:
from openai import OpenAI
from config import API_KEY, ASSISTANT_ID
client = OpenAI(api_key=API_KEY)

In [ ]:
completion = client.chat.completions.create(model = "gpt-3.5-turbo", messages = [get_user_prompt(user_prompt="what museums should I visit in Paris?")])
print(completion.choices[0].message.content)

Paris is home to some of the most famous and impressive museums in the world. Here are a few that you should definitely consider visiting:

1. The Louvre - This iconic museum is home to thousands of works of art, including the famous Mona Lisa. It is located at coordinates: {Louvre: 48.8606° N, 2.3376° E}.

2. Musée d'Orsay - Housed in a stunning former railway station, this museum is home to an impressive collection of Impressionist and Post-Impressionist art. It is located at coordinates: {Musée d'Orsay: 48.8599° N, 2.3266° E}.

3. Centre Pompidou - This modern art museum is known for its unique architecture and impressive collection of contemporary art. It is located at coordinates: {Centre Pompidou: 48.8606° N, 2.3521° E}.

4. Musée Rodin - This museum is dedicated to the works of the famous sculptor Auguste Rodin and is located in his former home and studio. It is located at coordinates: {Musée Rodin: 48.8554° N, 2.3155° E}.

5. Musée de l'Orangerie - This museum is known for its 

### Testing with Asistants

In [ ]:
#already created an assistant, checking list of assistants
my_assistants = client.beta.assistants.list(
    order="desc",
    limit="20",
)
print(my_assistants.data)

[Assistant(id='asst_ERzvBehgZHH410kiTOuGtlNJ', created_at=1710757402, description=None, file_ids=[], instructions='you are a travel guide, an expert in geography and tourism, well suited to inform travellers on geographical, historical and cultural data', metadata={}, model='gpt-3.5-turbo-0125', name='AliveBot', object='assistant', tools=[ToolCodeInterpreter(type='code_interpreter')])]


In [ ]:
my_assistant = client.beta.assistants.retrieve(assistant_id=ASSISTANT_ID)

In [ ]:
'''in case we want to modify the params'''

# my_updated_assistant = client.beta.assistants.update(
#   "asst_abc123",
#   instructions="You are an HR bot, and you have access to files to answer employee questions about company policies. Always response with info from either of the files.",
#   name="HR Helper",
#   tools=[{"type": "retrieval"}],
#   model="gpt-4",
#   file_ids=["file-abc123", "file-abc456"],
# )

### Experimenting with threads

In [3]:
#create an empty thread

empty_thread = client.beta.threads.create()
print(empty_thread)

Thread(id='thread_JpwIm8ehLphD0iLtykFiX4JA', created_at=1712524345, metadata={}, object='thread')


In [7]:
#create a user message within the thread

thread_message = client.beta.threads.messages.create(
  "thread_JpwIm8ehLphD0iLtykFiX4JA",
  role="user",
  content="Where is the Statue of Liberty?",
)
print(thread_message)

ThreadMessage(id='msg_MV0ClkWcangoGhPiR7lH9RMs', assistant_id=None, content=[MessageContentText(text=Text(annotations=[], value='Where is the Statue of Liberty?'), type='text')], created_at=1712524450, file_ids=[], metadata={}, object='thread.message', role='user', run_id=None, thread_id='thread_JpwIm8ehLphD0iLtykFiX4JA')


In [8]:
#running the thread with the agent

run = client.beta.threads.runs.create(
  thread_id="thread_JpwIm8ehLphD0iLtykFiX4JA",
  assistant_id=ASSISTANT_ID
)

In [9]:
run

Run(id='run_i367aKkhTbsvYcLQEZjK6rVp', assistant_id='asst_ERzvBehgZHH410kiTOuGtlNJ', cancelled_at=None, completed_at=None, created_at=1712524459, expires_at=1712525059, failed_at=None, file_ids=[], instructions='you are a travel guide, an expert in geography and tourism, well suited to inform travellers on geographical, historical and cultural data', last_error=None, metadata={}, model='gpt-3.5-turbo-0125', object='thread.run', required_action=None, started_at=None, status='queued', thread_id='thread_JpwIm8ehLphD0iLtykFiX4JA', tools=[ToolAssistantToolsCode(type='code_interpreter')], usage=None, temperature=1.0)

In [10]:
#retrieving the thread

my_thread = client.beta.threads.messages.list("thread_JpwIm8ehLphD0iLtykFiX4JA")
print(my_thread.data)

[ThreadMessage(id='msg_DmndEO7cAQGpnzsP8OX6pUj6', assistant_id='asst_ERzvBehgZHH410kiTOuGtlNJ', content=[MessageContentText(text=Text(annotations=[], value='The Statue of Liberty is located on Liberty Island in New York Harbor, in the United States of America. Liberty Island is part of the territory of the state of New York. The statue was a gift from the people of France to the United States and is a symbol of freedom and democracy. It is one of the most iconic landmarks in the world and a popular tourist attraction.'), type='text')], created_at=1712524460, file_ids=[], metadata={}, object='thread.message', role='assistant', run_id='run_i367aKkhTbsvYcLQEZjK6rVp', thread_id='thread_JpwIm8ehLphD0iLtykFiX4JA'), ThreadMessage(id='msg_MV0ClkWcangoGhPiR7lH9RMs', assistant_id=None, content=[MessageContentText(text=Text(annotations=[], value='Where is the Statue of Liberty?'), type='text')], created_at=1712524450, file_ids=[], metadata={}, object='thread.message', role='user', run_id=None, th

In [16]:
thread_message = client.beta.threads.messages.create(
  "thread_JpwIm8ehLphD0iLtykFiX4JA",
  role="user",
  content="Where is The White House",
)

In [17]:
run = client.beta.threads.runs.create(
  thread_id="thread_JpwIm8ehLphD0iLtykFiX4JA",
  assistant_id=ASSISTANT_ID
)

In [28]:
my_thread = client.beta.threads.messages.list("thread_JpwIm8ehLphD0iLtykFiX4JA").data
my_thread

[ThreadMessage(id='msg_geOC1kaDb07PQNwusS3CQP1w', assistant_id='asst_ERzvBehgZHH410kiTOuGtlNJ', content=[MessageContentText(text=Text(annotations=[], value='The White House is located at 1600 Pennsylvania Avenue NW in Washington, D.C., the capital city of the United States. It serves as the official residence and workplace of the President of the United States. The White House is an iconic symbol of American democracy and is a popular tourist destination for visitors from around the world.'), type='text')], created_at=1712524803, file_ids=[], metadata={}, object='thread.message', role='assistant', run_id='run_OOci2HFRS6BsJ9jzHhOad3nT', thread_id='thread_JpwIm8ehLphD0iLtykFiX4JA'),
 ThreadMessage(id='msg_uZn5CJrjPw826mJ0EQVpwYnC', assistant_id=None, content=[MessageContentText(text=Text(annotations=[], value='Where is The White House'), type='text')], created_at=1712524788, file_ids=[], metadata={}, object='thread.message', role='user', run_id=None, thread_id='thread_JpwIm8ehLphD0iLtykF

In [45]:
print(my_thread[1].content[0].text.value)
print(my_thread[0].content[0].text.value)

Where is The White House
The White House is located at 1600 Pennsylvania Avenue NW in Washington, D.C., the capital city of the United States. It serves as the official residence and workplace of the President of the United States. The White House is an iconic symbol of American democracy and is a popular tourist destination for visitors from around the world.


### Attempting Conversation

not using threads (model does not remember information)

In [10]:
def send_message_to_assistant(user_input):
    """
    Sends a message to an existing conversation thread with the Assistant and gets the response.
    """
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": user_input}
        ],
    )
    
    # Extract the Assistant's response from the created message
    assistant_response = response.choices[0].message.content  # Adjust based on actual response structure
    return assistant_response

In [11]:
def conversation():
    print("Chat with the Assistant! Type 'quit' to exit.")
    while True:
        user_input = input("You: ")
        if user_input.lower() == 'quit':
            break

        assistant_response = send_message_to_assistant(user_input)
        print("Assistant:", assistant_response)

In [12]:
conversation()

Chat with the Assistant! Type 'quit' to exit.
Assistant: Hello! How can I assist you today?
Assistant: Hello Bob! How can I assist you today?
Assistant: I am sorry, I do not know your name as I am an AI assistant and do not have access to personal information.
Assistant: 2 + 2 equals 4.
Assistant: Hello! How can I assist you today?


using threads

In [53]:
def create_empty_thread():
    empty_thread = client.beta.threads.create()
    return empty_thread.id

In [74]:
import os

def send_and_run():
    """
    Sends a user's message to the Assistant and retrieves the response within a thread.
    """

    id = create_empty_thread()
    print("Chat with the Assistant! Type 'quit' to exit.")

    while True:

        thread_data = client.beta.threads.messages.list(id).data
        for msg in thread_data[::-1]:
            print(msg)
        print('\n')

        os.system('cls')
        user_input = input("User: ")
        if user_input.lower() == 'quit':
            break

        # Sending a message to the thread
        thread_message = client.beta.threads.messages.create(
            id,
            role="user",
            content=user_input
        )

        # Running the thread with the specific assistant
        run = client.beta.threads.runs.create(
            id,
            assistant_id=ASSISTANT_ID
        )
        #return(thread_data[counter].content[0].text.value)

In [75]:
send_and_run()

Chat with the Assistant! Type 'quit' to exit.


ThreadMessage(id='msg_0G4ixQx14plBm80NIA2rkUSQ', assistant_id=None, content=[MessageContentText(text=Text(annotations=[], value='Hello'), type='text')], created_at=1712526846, file_ids=[], metadata={}, object='thread.message', role='user', run_id=None, thread_id='thread_vcYd1rXbr0lIvVOwKA4mTqBK')


ThreadMessage(id='msg_0G4ixQx14plBm80NIA2rkUSQ', assistant_id=None, content=[MessageContentText(text=Text(annotations=[], value='Hello'), type='text')], created_at=1712526846, file_ids=[], metadata={}, object='thread.message', role='user', run_id=None, thread_id='thread_vcYd1rXbr0lIvVOwKA4mTqBK')
ThreadMessage(id='msg_pjn57ggXgTD88XTKl2wZcEqG', assistant_id='asst_ERzvBehgZHH410kiTOuGtlNJ', content=[MessageContentText(text=Text(annotations=[], value='Hello! How can I assist you today?'), type='text')], created_at=1712526847, file_ids=[], metadata={}, object='thread.message', role='assistant', run_id='run_fRmzhhki46j4aGKX2XaiaAvc', thread_id='thr